In [202]:
# https://flinhong.com/2017/10/30/query-yahoo-finance-historical-data-via-python-requests/

import re
import sys
import time
import datetime
import requests
import pandas as pd


def get_cookie_value(r):
    return {'B': r.cookies['B']}

def get_page_data(symbol):
    url = "https://finance.yahoo.com/quote/%s/?p=%s" % (symbol, symbol)
    r = requests.get(url)
    cookie = get_cookie_value(r)
    lines = r.content.decode('unicode-escape').strip(). replace('}', '\n')
    return cookie, lines.split('\n')

def find_crumb_store(lines):
    for l in lines:
        if re.findall(r'CrumbStore', l):
            return l
    print("Did not find CrumbStore")

def split_crumb_store(v):
    return v.split(':')[2].strip('"')

def get_cookie_crumb(symbol):
    cookie, lines = get_page_data(symbol)
    crumb = split_crumb_store(find_crumb_store(lines))
    return cookie, crumb

def get_data(symbol, start_date, end_date, cookie, crumb):
    filename = '%s.csv' % (symbol)
    url = "https://query1.finance.yahoo.com/v7/finance/download/%s?period1=%s&period2=%s&interval=1d&events=history&crumb=%s" % (symbol, start_date, end_date, crumb)
    response = requests.get(url, cookies=cookie)
    return response

def get_now_epoch():
    return int(time.time())

def convert_to_dataframe(data, symbol):
    data = data.content.decode("utf-8").splitlines()
    data = [i.strip(',') for i in data]
    data = pd.DataFrame(data)
    data = data[0].str.split(',', expand=True)
    cols = data.iloc[0,:].values.tolist()
    data.columns = cols
    data = data[1:]
    data['symbol'] = symbol
    data['Date'] = pd.DatetimeIndex(data['Date'])
    data[['Open','High','Low','Close','Adj Close','Volume']] = data[['Open','High','Low','Close','Adj Close','Volume']].astype(float)
    return data

def download_quotes(symbols):

    if type(symbols)!=list:
        symbols = [symbols]
        
    start_date = 0
    end_date = get_now_epoch()
    
    stocks = []
    for symbol in symbols:
        cookie, crumb = get_cookie_crumb(symbol)
        data = get_data(symbol, start_date, end_date, cookie, crumb)
        stocks.append(convert_to_dataframe(data, symbol))
        
    return pd.concat(stocks, axis=0)


In [ ]:
#if in eider, comment out following code
%matplotlib inline

In [195]:
df = download_quotes(['^GSPC','AMZN'])
df.head()

<class 'list'> True
['^GSPC', 'AMZN']


/Users/dangoodb/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: invalid escape sequence '\/'


,Date,Open,High,Low,Close,Adj Close,Volume,symbol
1,1970-01-02,92.059998,93.540001,91.790001,93.000000,93.000000,8050000.0,^GSPC
2,1970-01-05,93.000000,94.250000,92.529999,93.459999,93.459999,11490000.0,^GSPC
3,1970-01-06,93.459999,93.809998,92.129997,92.820000,92.820000,11460000.0,^GSPC
4,1970-01-07,92.820000,93.379997,91.930000,92.629997,92.629997,10010000.0,^GSPC
5,1970-01-08,92.629997,93.470001,91.989998,92.680000,92.680000,10670000.0,^GSPC


# Your Turn!


1\. What was the "Open" price of Amazon stock (AMZN) on February 15, 2012?

2\. What was the highest volume of Amazon stock traded in one day?


3\. On what day day did this occur?

4\. What was Amazon's biggest one day stock growth in dollar terms? in percentage terms?

5\. Create line chart showing Amazon stock price over time

6\. Create line chart showing average monthly stock price of Amazon (AMZN) vs S&P 500 (^GSPC) since January 2000 using Adj Close

hint: df['column'].dt.to_period("M") can convert a date to a month

7\. Create same chart in 2) but normalize it so that both stocks are set to 1.0 in January 2000